In [2]:
from fusion import compute_bdi, compute_ec,compute_st, process_vibration, process_ae

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt 
import scipy
import os
import glob
import seaborn as sns
import itertools
import gc
import time
import scienceplots
import sys
import librosa
from nptdms import TdmsFile
from scipy import stats
from natsort import natsorted

from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from matplotlib.ticker import FormatStrFormatter
import matplotlib as mpl
import seaborn as sns
from scipy.stats import norm
from scipy.interpolate import interp1d
from scipy.interpolate import make_interp_spline, BSpline

sys.path.append('../utils/')
from utils.preprocessing import centimeter,one_column,two_column,cm_std,cm_bright,cm_highCon,cm_mark
np.random.seed(16)
matplotlib.rcParams['text.usetex'] = True
plt.rcParams["text.usetex"] = True
matplotlib.rcParams['figure.dpi'] = 300
plt.style.use(['science','nature'])
plt.rcParams['figure.constrained_layout.use'] = True
mpl.rcParams.update(one_column)

import sys
sys.path.append('./utils')
# from pydub import AudioSegment
import itertools
import string
import glob
import subprocess
import seedir
from utils.preprocessing import print_tdms_structure,check_identical_csv_lengths
from utils.preprocessing import linearSpectrogram, logMelSpectrogram,melSpectrogram, logSpectrogram, standardize_array,slice_indices

alphabet = list(string.ascii_lowercase)
sampling_rate_ae = 4*1e6
sampling_rate_vib = 51.2*1e3

project_name = ["Grinding","XiAnJiaoTong"]

if os.name == 'posix':
    project_dir:str = os.path.join(
    subprocess.getoutput("echo $DATADIR"),
    *project_name,
    )
elif os.name == 'nt':
    project_dir:str = os.path.join(
    subprocess.getoutput("echo %datadir%"),
    *project_name
    )

dataDir_ae = os.path.join(project_dir,"AE")
dataDir_vib = os.path.join(project_dir,"Vibration")

In [17]:
df = pd.read_excel(os.path.join(project_dir,"parameters.xlsx"),index_col=0)
df.columns = ["Surface speed","Workpiece rotation speed","Grinding depth"]
df["Surface speed"] = df["Surface speed"].str.extract(r'(\d+)').astype(float) # mm/s
df["Workpiece rotation speed"] = df["Workpiece rotation speed"].str.extract(r'(\d+)').astype(float) # rpm
df["Grinding depth"] = df["Grinding depth"].str.extract(r'(\d+)').astype(float) # um
parameters = df.copy()
del df

df = pd.read_csv(os.path.join(project_dir,"surface roughness.csv"),index_col=None)
df.columns=["Surface roughness"]
sr = df.iloc[:,0].to_numpy()
parameters

,Surface speed,Workpiece rotation speed,Grinding depth
No.,,,
1,25.0,20.0,20.0
2,25.0,40.0,30.0
3,25.0,60.0,40.0
4,25.0,80.0,50.0
5,30.0,40.0,40.0
6,30.0,60.0,50.0
7,30.0,80.0,20.0
8,30.0,20.0,30.0
9,35.0,60.0,50.0


In [3]:
ae_names = natsorted([os.path.join(dataDir_ae,i) for i in os.listdir(dataDir_ae) if os.path.splitext(i)[1] == '.txt'])
ae_names

['C:\\Users\\Public\\Documents\\Data\\Grinding\\XiAnJiaoTong\\AE\\1-01-0.txt',
 'C:\\Users\\Public\\Documents\\Data\\Grinding\\XiAnJiaoTong\\AE\\1-02-0.txt',
 'C:\\Users\\Public\\Documents\\Data\\Grinding\\XiAnJiaoTong\\AE\\1-03-0.txt',
 'C:\\Users\\Public\\Documents\\Data\\Grinding\\XiAnJiaoTong\\AE\\1-04-0.txt',
 'C:\\Users\\Public\\Documents\\Data\\Grinding\\XiAnJiaoTong\\AE\\1-05-0.txt',
 'C:\\Users\\Public\\Documents\\Data\\Grinding\\XiAnJiaoTong\\AE\\1-06-0.txt',
 'C:\\Users\\Public\\Documents\\Data\\Grinding\\XiAnJiaoTong\\AE\\1-07-0.txt',
 'C:\\Users\\Public\\Documents\\Data\\Grinding\\XiAnJiaoTong\\AE\\1-08-0.txt',
 'C:\\Users\\Public\\Documents\\Data\\Grinding\\XiAnJiaoTong\\AE\\1-09-0.txt',
 'C:\\Users\\Public\\Documents\\Data\\Grinding\\XiAnJiaoTong\\AE\\1-10-0.txt',
 'C:\\Users\\Public\\Documents\\Data\\Grinding\\XiAnJiaoTong\\AE\\1-11-0.txt',
 'C:\\Users\\Public\\Documents\\Data\\Grinding\\XiAnJiaoTong\\AE\\1-12-0.txt',
 'C:\\Users\\Public\\Documents\\Data\\Grinding\\XiAn

In [42]:
_fn = os.path.split(ae_names[0])[1].split(".")[0]

p_n = int(_fn.split("-")[0])-1 # parameters index
v_s = parameters.iloc[p_n]['Surface speed']
v_w = parameters.iloc[p_n]['Workpiece rotation speed']
a_p = parameters.iloc[p_n]['Grinding depth']

In [18]:
vib_data = TdmsFile.read(os.path.join(dataDir_vib,f"{_fn[:-2]}","Test_Data.tdms")).groups()
vib_x = vib_data[0].channels()[0].data[:]
vib_y = vib_data[0].channels()[1].data[:]
vib_z = vib_data[0].channels()[2].data[:]

In [19]:
ae_df = pd.read_csv(ae_names[0],sep='\s',header=None)

C:\Users\mingwucn\AppData\Local\Temp\ipykernel_13756\3754738353.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ae_df = pd.read_csv(ae_names[0],sep='\s',header=None)


In [22]:
ae_indices = slice_indices(len(ae_df),int(sampling_rate_ae*0.01),0.1)

In [ ]:
_i0,_it = ae_indices[0]